<a href="https://colab.research.google.com/github/thegallier/dspy/blob/main/dspy_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U dspy-ai
!pip install -U openai jinja2
!pip install -U langchain langchain-community langchain-openai langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
import dspy

colbertv2 = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")

In [4]:
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

set_llm_cache(SQLiteCache(database_path="cache.db"))

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)


def retrieve(inputs):
    return [doc["text"] for doc in colbertv2(inputs["question"], k=5)]

In [ ]:
colbertv2("cycling")

[{'text': 'Cycling | Cycling, also called bicycling or biking, is the use of bicycles for transport, recreation, exercise or sport. Persons engaged in cycling are referred to as "cyclists", "bikers", or less commonly, as "bicyclists". Apart from two-wheeled bicycles, "cycling" also includes the riding of unicycles, tricycles, quadracycles, recumbent and similar human-powered vehicles (HPVs).',
  'pid': 2201868,
  'rank': 1,
  'score': 27.078739166259766,
  'prob': 0.3544841299722533,
  'long_text': 'Cycling | Cycling, also called bicycling or biking, is the use of bicycles for transport, recreation, exercise or sport. Persons engaged in cycling are referred to as "cyclists", "bikers", or less commonly, as "bicyclists". Apart from two-wheeled bicycles, "cycling" also includes the riding of unicycles, tricycles, quadracycles, recumbent and similar human-powered vehicles (HPVs).'},
 {'text': 'Cycling (ice hockey) | In ice hockey, cycling is an offensive strategy that moves the puck along 

In [ ]:
# From LangChain, import standard modules for prompting.
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Just a simple prompt for this task. It's fine if it's complex too.
prompt = PromptTemplate.from_template(
    "Given {context}, answer the question `{question}` as a tweet."
)

# This is how you'd normally build a chain with LCEL. This chain does retrieval then generation (RAG).
vanilla_chain = (
    RunnablePassthrough.assign(context=retrieve) | prompt | llm | StrOutputParser()
)

In [ ]:
# From DSPy, import the modules that know how to interact with LangChain LCEL.
from dspy.predict.langchain import LangChainModule, LangChainPredict

# This is how to wrap it so it behaves like a DSPy program.
# Just Replace every pattern like `prompt | llm` with `LangChainPredict(prompt, llm)`.
zeroshot_chain = (
    RunnablePassthrough.assign(context=retrieve)
    | LangChainPredict(prompt, llm)
    | StrOutputParser()
)
# Now we wrap it in LangChainModule
zeroshot_chain = LangChainModule(
    zeroshot_chain
)  # then wrap the chain in a DSPy module.



AttributeError: 'FieldInfo' object has no attribute 'finalize'

In [ ]:
### without langchain https://github.com/skandavivek/DSPy-blog

In [5]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [6]:
class fs(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        #self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question,context):
        #context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [8]:
!git clone https://github.com/skandavivek/transformerQA-finetuning.git

Cloning into 'transformerQA-finetuning'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 2.41 MiB | 3.18 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [9]:
import pandas as pd
df = pd.read_csv('/content/transformerQA-finetuning/train.csv')

In [10]:
#as you can see, there are 2 types of answers - one is ANSWERNOTFOUND, the other is when the extracted answer is present
df['human_ans_spans'].astype(str)

0                                      ANSWERNOTFOUND
1                                      ANSWERNOTFOUND
2                                      ANSWERNOTFOUND
3                            this show is OUTSTANDING
4       The costume design by Susan Matheson is great
                            ...                      
2496                                   ANSWERNOTFOUND
2497                                   ANSWERNOTFOUND
2498                                   ANSWERNOTFOUND
2499                                   ANSWERNOTFOUND
2500                                   ANSWERNOTFOUND
Name: human_ans_spans, Length: 2501, dtype: object

In [11]:
df1=df.loc[df['human_ans_spans']=='ANSWERNOTFOUND'].reset_index(drop=True)
df2=df.loc[df['human_ans_spans']!='ANSWERNOTFOUND'].reset_index(drop=True)

In [12]:
df1=df1[:4][['question','review','human_ans_spans']]
df2=df2[:4][['question','review','human_ans_spans']]

df1.columns=['question','context','answer']

df2.columns=['question','context','answer']
df_c=pd.concat([df1,df2]).sort_index().reset_index(drop=True)
df_c

,question,context,answer
0,Who is the author of this series?,Whether it be in her portrayal of a nerdy lesb...,ANSWERNOTFOUND
1,Is this series good and excelent?,"At the time of my review, there had been 910 c...",this show is OUTSTANDING
2,Can we enjoy the movie along with our family ?,"An outstanding romantic comedy, 13 Going on 30...",ANSWERNOTFOUND
3,How is the costume design?,"""Fright Night"" is great! This is how the story...",The costume design by Susan Matheson is great
4,Does this one good?,"To let the truth be known, I watched this movi...",ANSWERNOTFOUND
5,What criticism deserves the movie Passion of C...,Revenge of the Sith is for children and for ad...,Oh my god is this a STUPID film
6,How are the special effects?,As with all Star Wars (or science fiction) mov...,ANSWERNOTFOUND
7,How is the scene?,The best KingKong movie ever made in my opinio...,KingKong


In [14]:
import json
qa_pairs = json.loads(df_c.to_json(orient="records"))

In [15]:
# Create dataset
dataset = [dspy.Example(x).with_inputs('question','context') for x in qa_pairs]

trainset = [x.with_inputs('question','context') for x in dataset[:4]]
devset = [x.with_inputs('question','context') for x in dataset[4:]]

In [16]:
train_example = trainset[0]
print(f"Question: {train_example.question}")
print(f"Context: {train_example.context}")
print(f"Answer: {train_example.answer}")

Question: Who is the author of this series?
Context: Whether it be in her portrayal of a nerdy lesbian or a punk rock rebel, Maslany's plural personalities, (though very stereotypical), are entertaining eye-candy. Combined with a complex and unpredictable plot line, this show is surprisingly addictive. ANSWERNOTFOUND
Answer: ANSWERNOTFOUND


In [17]:
dev_example = devset[0]

In [18]:
print(f"For this dataset, training examples have input keys {train_example.inputs().keys()} and label keys {train_example.labels().keys()}")
print(f"For this dataset, dev examples have input keys {dev_example.inputs().keys()} and label keys {dev_example.labels().keys()}")

For this dataset, training examples have input keys ['question', 'context'] and label keys ['answer']
For this dataset, dev examples have input keys ['question', 'context'] and label keys ['answer']


In [50]:
!pip install -U dspy

In [52]:
import dspy

In [54]:
import logging
from typing import Any

import backoff

try:
    import groq
    from groq import Groq

    groq_api_error = (groq.APIError, groq.RateLimitError)
except ImportError:
    groq_api_error = Exception


from dsp.modules.lm import LM


def backoff_hdlr(details):
    """Handler from https://pypi.org/project/backoff/"""
    print(
        "Backing off {wait:0.1f} seconds after {tries} tries "
        "calling function {target} with kwargs "
        "{kwargs}".format(**details),
    )


class GroqLM(LM):
    """Wrapper around groq's API.

    Args:
        model (str, optional): groq supported LLM model to use. Defaults to "mixtral-8x7b-32768".
        api_key (Optional[str], optional): API provider Authentication token. use Defaults to None.
        **kwargs: Additional arguments to pass to the API provider.
    """

    def __init__(
        self,
        api_key: str,
        model: str = "mixtral-8x7b-32768",
        **kwargs,
    ):
        super().__init__(model)
        self.provider = "groq"
        if api_key:
            self.api_key = api_key
            self.client = Groq(api_key=api_key)
        else:
            raise ValueError("api_key is required for groq")

        self.kwargs = {
            "temperature": 0.0,
            "max_tokens": 150,
            "top_p": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            "n": 1,
            **kwargs,
        }
        models = self.client.models.list().data
        if models is not None:
            if model in [m.id for m in models]:
                self.kwargs["model"] = model
        self.history: list[dict[str, Any]] = []

    def log_usage(self, response):
        """Log the total tokens from the Groq API response."""
        usage_data = response.usage  # Directly accessing the 'usage' attribute
        if usage_data:
            total_tokens = usage_data.total_tokens
            logging.debug(f"Groq Total Tokens Response Usage: {total_tokens}")

    def basic_request(self, prompt: str, **kwargs):
        raw_kwargs = kwargs

        kwargs = {**self.kwargs, **kwargs}

        kwargs["messages"] = [{"role": "user", "content": prompt}]
        response = self.chat_request(**kwargs)

        history = {
            "prompt": prompt,
            "response": response.choices[0].message.content,
            "kwargs": kwargs,
            "raw_kwargs": raw_kwargs,
        }

        self.history.append(history)

        return response

    @backoff.on_exception(
        backoff.expo,
        groq_api_error,
        max_time=1000,
        on_backoff=backoff_hdlr,
    )
    def request(self, prompt: str, **kwargs):
        """Handles retreival of model completions whilst handling rate limiting and caching."""
        if "model_type" in kwargs:
            del kwargs["model_type"]

        return self.basic_request(prompt, **kwargs)

    def _get_choice_text(self, choice) -> str:
        return choice.message.content

    def chat_request(self, **kwargs):
        """Handles retreival of model completions whilst handling rate limiting and caching."""
        response = self.client.chat.completions.create(**kwargs)
        return response

    def __call__(
        self,
        prompt: str,
        only_completed: bool = True,
        return_sorted: bool = False,
        **kwargs,
    ) -> list[dict[str, Any]]:
        """Retrieves completions from model.

        Args:
            prompt (str): prompt to send to model
            only_completed (bool, optional): return only completed responses and ignores completion due to length. Defaults to True.
            return_sorted (bool, optional): sort the completion choices using the returned probabilities. Defaults to False.

        Returns:
            list[dict[str, Any]]: list of completion choices
        """

        assert only_completed, "for now"
        assert return_sorted is False, "for now"
        response = self.request(prompt, **kwargs)

        self.log_usage(response)

        choices = response.choices

        completions = [self._get_choice_text(c) for c in choices]
        if return_sorted and kwargs.get("n", 1) > 1:
            scored_completions = []

            for c in choices:
                tokens, logprobs = (
                    c["logprobs"]["tokens"],
                    c["logprobs"]["token_logprobs"],
                )

                if "<|endoftext|>" in tokens:
                    index = tokens.index("<|endoftext|>") + 1
                    tokens, logprobs = tokens[:index], logprobs[:index]

                avglog = sum(logprobs) / len(logprobs)
                scored_completions.append((avglog, self._get_choice_text(c)))

            scored_completions = sorted(scored_completions, reverse=True)
            completions = [c for _, c in scored_completions]

        return completions

In [60]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00


In [63]:
from groq import Groq

In [64]:
lm = GroqLM(model='mixtral-8x7b-32768', api_key ="" )

In [67]:
lm("say hi")

["Hello! It's nice to meet you. Is there something specific you would like to talk about or ask me? I'm here to help with any questions you might have. Just let me know how I can assist you."]

In [65]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo')
turbo = dspy.OpenAI(model='gpt-4-turbo')
turbo = dspy.OpenAI(model=lm)

dspy.settings.configure(lm=turbo)

TypeError: argument of type 'GroqLM' is not iterable

In [37]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question=dev_example.question,context = dev_example.context)

# Print the input and the prediction.
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

Question: Does this one good?
Predicted Answer: Question: Does this one good?
Answer: Unclear question.


In [38]:
turbo.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: Does this one good?
Answer: Question: Does this one good?
Answer: Unclear question.





In [39]:
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=dev_example.question)

# Print the input, the chain of thought, and the prediction.
print(f"Question: {dev_example.question}")
print(f"Thought: {pred.rationale.split(':', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Question: Does this one good?
Thought: Does this one good?
Reasoning: Let's think step by step in order to understand the question. The question seems to be grammatically incorrect or incomplete. It likely intends to ask if something is good or performs well.
Predicted Answer: Clarify question


In [25]:
pred.rationale

'Answer: Yes'

In [40]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [41]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [42]:
class fs(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        #self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question,context):
        #context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [43]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    #answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM

# Set up a basic teleprompter, which will compile our program.
teleprompter = BootstrapFewShot(metric=validate_answer)

# Compile!
compiled_fs = teleprompter.compile(fs(), trainset=trainset)

100%|██████████| 4/4 [00:12<00:00,  3.20s/it]

Bootstrapped 0 full traces after 4 examples in round 0.


In [44]:
# Ask any question you like to this simple few-shot program.


my_question = "Do you like Avocados?"
context = dev_example.context

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_fs(my_question,context)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts: {pred.context}")

Question: Do you like Avocados?
Predicted Answer: ANSWERNOTFOUND
Retrieved Contexts: To let the truth be known, I watched this movie with a mix of anticipation and fear. Being an avid Star Wars fan, I was excited to see any Star Wars movie, but I suspected this would be as disappointing as the Phantom Menace. WRONG! Although this doesn't even come close to the great casting and story lines and sheer art of the first three Star Wars series, it was WAY better than Phantom Menace for the following reasons: 1) This movie included LESS Jar-Jar, which, despite initial heavy marketing for the first movie, the character was found by the general consensus to be REALLY annoying. 2) This movie demonstrated some of the political turmoil behind the original Star Wars movies. 3) You get to see some of what led Anakin to turn over to the Dark Side. Finally, the special effects were really good!It was not 4 or 5 stars because the actors that were cast in this movie (as well as The Phantom Menace) are 

In [31]:
turbo.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context: "Fright Night" is great! This is how the story goes: Senior Charley Brewster finally has it all -- he's running with the popular crowd and dating the hottest girl in high school. In fact, he's so cool he's even dissing his best friend Ed. But trouble arrives when an intriguing stranger Jerry moves in next door. He seems like a great guy at first, but there's something not quite right -- and everyone, including Charley's mom, doesn't notice. After witnessing some very unusual activity, Charley comes to an unmistakable conclusion: Jerry is a vampire preying on his neighborhood. Unable to convince anyone that he's telling the truth, Charley has to find a way to get rid of the monster himself.The cast led by Anton Yelch

In [45]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    #answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM

# Set up a basic teleprompter, which will compile our program.
teleprompter = BootstrapFewShot(metric=validate_answer)

# Compile!
compiled_fs = teleprompter.compile(fs(), trainset=trainset)

100%|██████████| 4/4 [00:00<00:00, 1002.52it/s]

Bootstrapped 0 full traces after 4 examples in round 0.


In [46]:
pred = compiled_fs(my_question,context)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts: {pred.context}")

Question: Do you like Avocados?
Predicted Answer: ANSWERNOTFOUND
Retrieved Contexts: To let the truth be known, I watched this movie with a mix of anticipation and fear. Being an avid Star Wars fan, I was excited to see any Star Wars movie, but I suspected this would be as disappointing as the Phantom Menace. WRONG! Although this doesn't even come close to the great casting and story lines and sheer art of the first three Star Wars series, it was WAY better than Phantom Menace for the following reasons: 1) This movie included LESS Jar-Jar, which, despite initial heavy marketing for the first movie, the character was found by the general consensus to be REALLY annoying. 2) This movie demonstrated some of the political turmoil behind the original Star Wars movies. 3) You get to see some of what led Anakin to turn over to the Dark Side. Finally, the special effects were really good!It was not 4 or 5 stars because the actors that were cast in this movie (as well as The Phantom Menace) are 

In [47]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class fs(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        #self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question,context):
        #context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    #answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_answer)

# Compile!
compiled_fs = teleprompter.compile(fs(), trainset=trainset)

100%|██████████| 4/4 [00:00<00:00, 878.89it/s]

Bootstrapped 0 full traces after 4 examples in round 0.


In [48]:
pred = compiled_fs(my_question,context)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts: {pred.context}")

Question: Do you like Avocados?
Predicted Answer: ANSWERNOTFOUND
Retrieved Contexts: To let the truth be known, I watched this movie with a mix of anticipation and fear. Being an avid Star Wars fan, I was excited to see any Star Wars movie, but I suspected this would be as disappointing as the Phantom Menace. WRONG! Although this doesn't even come close to the great casting and story lines and sheer art of the first three Star Wars series, it was WAY better than Phantom Menace for the following reasons: 1) This movie included LESS Jar-Jar, which, despite initial heavy marketing for the first movie, the character was found by the general consensus to be REALLY annoying. 2) This movie demonstrated some of the political turmoil behind the original Star Wars movies. 3) You get to see some of what led Anakin to turn over to the Dark Side. Finally, the special effects were really good!It was not 4 or 5 stars because the actors that were cast in this movie (as well as The Phantom Menace) are 